## Load Foam Files

In [ ]:
import os
import pyvista as pv
import vtk
import igl

base_dir = r"Z:\Zack\Data\Incoming\Cat_parametric_study\High_Fidelity"

# Dictionary to store each loaded mesh
cases = {}

# Loop over RUN001 to RUN020 (adjust range if needed)
for i in range(1, 21):
    folder_name = f"RUN{i:03d}"  # e.g. RUN001, RUN002, ...
    foam_file = os.path.join(base_dir, folder_name, "case.foam")

    # Check if the foam file actually exists
    if os.path.exists(foam_file):
        reader = pv.OpenFOAMReader(foam_file)
        mesh = reader.read()
        cases[folder_name] = mesh
        print(f"Loaded {foam_file}")
    else:
        print(f"File not found: {foam_file}")

# 'cases' now holds the meshes from RUN001 to RUN020
# Each mesh can be accessed via, e.g., cases["RUN001"]
# You can now do further processing or visualization.



Loaded Z:\Zack\Data\Incoming\Cat_parametric_study\High_Fidelity\RUN001\case.foam
Loaded Z:\Zack\Data\Incoming\Cat_parametric_study\High_Fidelity\RUN002\case.foam
Loaded Z:\Zack\Data\Incoming\Cat_parametric_study\High_Fidelity\RUN003\case.foam
Loaded Z:\Zack\Data\Incoming\Cat_parametric_study\High_Fidelity\RUN004\case.foam


KeyboardInterrupt: 

In [ ]:
import vtk 
import igl 
import numpy as np

# Load the OpenFOAM case file
reader = vtk.vtkOpenFOAMReader()
# reader.SetFileName("Z:\Zack\Data\Incoming\Cat_parametric_study\High_Fidelity\RUN001\case.foam")
reader.SetFileName('C:/Users/zack/Desktop/RUN001/case.foam')
# reader.SetFileName("Z:\Zack\Data\Incoming\Cat_parametric_study\Low_Fidelity\RUN001\case.foam")7
reader.DecomposePolyhedraOn()
reader.Update()

# Select only the 'internalMesh' region
reader.DisableAllPatchArrays()  # Disable all other patches
reader.SetPatchArrayStatus("internalMesh",1)  # Enable internalMesh
reader.SetPatchArrayStatus("patch/hull", 1)  # Enable hull mesh (low curvature elements)
reader.SetPatchArrayStatus("patch/hull_hc",1)  # Enable hull mesh (high curvature elements)
reader.Update()

# Step 2: Get the multi-block dataset
output = reader.GetOutput()

# Toggle to True if you want LSCM instead of Harmonic
LSCM = False

############################
# OPTIONAL: Clip the top
############################
CLIP_TOP = True
clip_z_offset = 0.95  # how much to lower the clipping plane

# Load hull surface mesh
hull_blocks = output.GetBlock(1)
hull_meshes = []

for i in range(hull_blocks.GetNumberOfBlocks()):
    sub_block = hull_blocks.GetBlock(i)
    if isinstance(sub_block, vtk.vtkUnstructuredGrid):
        poly = pv.wrap(sub_block).extract_surface().triangulate()
        hull_meshes.append(poly)
    elif isinstance(sub_block, vtk.vtkPolyData):
        poly = pv.wrap(sub_block).triangulate()
        # Convert cell-based pressure to point-based if needed
        if "p" in poly.cell_data:
            poly = poly.cell_data_to_point_data()
        hull_meshes.append(poly)
        


# Combine & surface
hull_mesh = pv.MultiBlock(hull_meshes).combine().extract_surface().triangulate()

# Optionally clip the top
if CLIP_TOP:
    z_max = hull_mesh.points[:, 2].max()
    clip_z_max = z_max - clip_z_offset  # user offset
    clip_plane = vtk.vtkPlane()
    clip_plane.SetOrigin(0, 0, clip_z_max)
    clip_plane.SetNormal(0, 0, -1)
    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(hull_mesh)
    clipper.SetClipFunction(clip_plane)
    clipper.Update()
    hull_mesh = pv.wrap(clipper.GetOutput())

# Compute cell areas & threshold small triangles
hull_mesh["Area"] = hull_mesh.compute_cell_sizes()["Area"]
hull_mesh = hull_mesh.threshold(value=1e-5, scalars="Area")
hull_mesh = hull_mesh.clean()

# Check for non-manifold edges
feature_edges = vtk.vtkFeatureEdges()
feature_edges.SetInputData(hull_mesh)
feature_edges.BoundaryEdgesOn()
feature_edges.NonManifoldEdgesOn()
feature_edges.ManifoldEdgesOff()
feature_edges.FeatureEdgesOff()
feature_edges.Update()
nonmanifold_edges = feature_edges.GetOutput()
print("Non-manifold edge count:", nonmanifold_edges.GetNumberOfCells())

# Extract geometry
v = hull_mesh.points.astype(np.float64)
if hull_mesh.n_cells == 0:
    raise ValueError("No faces found in hull_mesh after triangulation.")
f = hull_mesh.cells.reshape(-1, 4)[:, 1:].astype(np.int32)

# Extract scalar "p"
if "p" not in hull_mesh.point_data:
    raise ValueError("Scalar field 'p' not found!")
scalar = hull_mesh.point_data["p"]

# # Normalize
# if np.min(scalar) != np.max(scalar):
#     scalar = (scalar - np.min(scalar)) / (np.max(scalar) - np.min(scalar))
# else:
#     scalar = np.ones(len(scalar))

############################
# Param: LSCM vs. Harmonic
############################
bnd = igl.boundary_loop(f)

if LSCM:
    # LSCM with two anchor points
    boundary_coords = v[bnd]
    bow_idx_local = np.argmin(boundary_coords[:, 0])  # min X
    stern_idx_local = np.argmax(boundary_coords[:, 0])  # max X
    bow_idx = bnd[bow_idx_local]
    stern_idx = bnd[stern_idx_local]

    b = np.array([bow_idx, stern_idx])
    bc = np.array([[0.0, 0.0], [1.0, 0.0]])

    # Attempt LSCM
    print("Using LSCM for mapping...")
    _, uv = igl.lscm(v, f, b, bc)
    if uv.shape[0] == 0:
        raise RuntimeError("LSCM returned an empty UV map.")
else:
    # Circle boundary + Harmonic
    print("Using circle boundary + harmonic mapping...")
    bnd_uv = igl.map_vertices_to_circle(v, bnd)  # place boundary on circle
    uv = igl.harmonic(v, f, bnd, bnd_uv.astype(v.dtype), 1)

# Verify success
if uv.shape[0] != v.shape[0]:
    raise ValueError(f"UV mapping failed: uv has {uv.shape[0]} points, expected {v.shape[0]}")
print("UV mapping success. uv shape:", uv.shape)

############################
# Visualize in PyVista
############################
uv_points = np.column_stack((uv, np.zeros(uv.shape[0])))
faces_pv = np.hstack((np.full((f.shape[0], 1), 3), f)).astype(np.int32)

uv_mesh = pv.PolyData(uv_points, faces_pv)
uv_mesh.point_data["p"] = scalar

# probe points to plot 
idx_bow = np.argmin(hull_mesh.points[:, 0])  # min X for bow
print("3D Pressure at bow:", hull_mesh['p'][idx_bow])
print("UV Pressure at bow:", uv_mesh['p'][idx_bow])

# Subplot to compare original vs. UV
plotter = pv.Plotter(shape=(1,2))

# Left: 3D hull with pressure
plotter.subplot(0,0)
clim_3d = [scalar.min(), scalar.max()]
# clim_3d =  [-1000, 20000]
plotter.add_mesh(hull_mesh, scalars=scalar, clim=clim_3d, cmap="coolwarm", show_edges=False)
plotter.add_points(hull_mesh.points[idx_bow], style='points', cmap = 'blue', point_size=10)
plotter.add_text("Original Hull with p", font_size=10)

# Right: 2D UV mesh
plotter.subplot(0,1)
clim_uv = [scalar.min(), scalar.max()]
# clim_3d =  [-1000, 20000]
plotter.add_mesh(uv_mesh, scalars="p", clim=clim_uv, cmap="coolwarm", show_edges=True)
plotter.add_points(uv_mesh.points[idx_bow], style='points', cmap = 'blue', point_size=10)
plotter.add_text("UV Map with p", font_size=10)


plotter.show()

ERROR:root:Input for connection index 0 on input port index 0 for algorithm vtkFeatureEdges (000001D163C2C980) is of type vtkUnstructuredGrid, but a vtkPolyData is required.


Non-manifold edge count: 0
Using circle boundary + harmonic mapping...
UV mapping success. uv shape: (470410, 2)
3D Pressure at bow: -2.0719278
UV Pressure at bow: -2.0719278


Widget(value='<iframe src="http://localhost:60116/index.html?ui=P_0x1d253d97920_2&reconnect=auto" class="pyvis…

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-457' coro=<WebSocketWriter.ping() done, defined at c:\Users\zack\AppData\Local\anaconda3\Lib\site-packages\aiohttp\http_websocket.py:711> exception=ConnectionResetError('Cannot write to closing transport')>
Traceback (most recent call last):
  File "c:\Users\zack\AppData\Local\anaconda3\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\zack\AppData\Local\anaconda3\Lib\site-packages\aiohttp\http_websocket.py", line 715, in ping
    await self._send_frame(message, WSMsgType.PING)
  File "c:\Users\zack\AppData\Local\anaconda3\Lib\site-packages\aiohttp\http_websocket.py", line 682, in _send_frame
    self._write(header + message)
  File "c:\Users\zack\AppData\Local\anaconda3\Lib\site-packages\aiohttp\http_websocket.py", line 702, in _write
    raise ConnectionResetError("Cannot write to closing transport")
ConnectionRes